In [24]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import cohen_kappa_score
from textblob import TextBlob
import language_check
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
import re
from gensim.models import Word2Vec
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score

stopwords = set(stopwords.words('english'))

In [25]:
df = pd.read_excel('features.xlsx')
df.head(2)

,essay_id,essay_set,essay,domain1_score,word_count,sentence_count,avg_word_length,num_exclamation_marks,num_question_marks,num_stopwords,...,verb_count,foreign_count,adj_count,conj_count,adv_count,beauty_score,maturity_score,vocab,sentiment_essay,Grammar_check
0,1,1,"Dear local newspaper, I think effects computer...",6.666667,338,16,4.550296,4,2,168,...,39,0,13,69,22,175.314893,0.029508,183,0.310471,11
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",7.500000,419,20,4.463007,1,1,189,...,56,0,14,80,21,239.987278,0.023986,217,0.274000,19


In [47]:
Set=[]
for i in range(1,9):
    Set.append(df[df['essay_set']==i])
print (Set)

[      essay_id  essay_set                                              essay  \
0            1          1  Dear local newspaper, I think effects computer...   
1            2          1  Dear @CAPS1 @CAPS2, I believe that using compu...   
2            3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...   
3            4          1  Dear Local Newspaper, @CAPS1 I have found that...   
4            5          1  Dear @LOCATION1, I know having computers has a...   
5            6          1  Dear @LOCATION1, I think that computers have a...   
6            7          1  Did you know that more and more people these d...   
7            8          1  @PERCENT1 of people agree that computers make ...   
8            9          1  Dear reader, @ORGANIZATION1 has had a dramatic...   
9           10          1  In the @LOCATION1 we have the technology of a ...   
10          11          1  Dear @LOCATION1, @CAPS1 people acknowledge the...   
11          12          1  Dear @CAPS1 

In [28]:
from sklearn import metrics
MSE_LR=[]
Log_R=[]
for data in Set:
    x = data.drop(['domain1_score','essay_id', 'essay','essay_set'], axis=1)

#   y = df['domain1_score']
#   x = df_normalized.drop(['domain1_score'],axis=1)
# # df['A']=df['A'].fillna(0.0).astype(int)
    y = data['domain1_score'].fillna(0.0).astype(int)
#     print (x)
#     print (y)
    x = np.array(x)
    y = np.array(y)
    where_are_NaNs = np.isnan(x)
    x[where_are_NaNs] = 0
    where_are_NaNs = np.isnan(y)
    y[where_are_NaNs] = 0
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    for train_index, test_index in kfold.split(x, y):
        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
    logistic_reg = LogisticRegression()
    logistic_reg.fit(X_train, y_train)
    y_pred = logistic_reg.predict(X_test)
#     print('\nLogistic regression classifier accuracy:', logistic_reg.score(X_test, y_test))
    MSE_LR.append( round(metrics.mean_squared_error(y_test,y_pred),4) )
    Log_R.append( round(logistic_reg.score(X_test, y_test),4) )
print ("Logistic Regression Accuracy for each set: ",Log_R)
print ("Linear Regression MSE for each set: ",MSE_LR)

/home/ayushi/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Logistic Regression Accuracy for each set:  [0.49009999999999998, 0.4874, 0.63270000000000004, 0.6119, 0.63970000000000005, 0.59219999999999995, 0.37859999999999999, 0.51060000000000005]
Linear Regression MSE for each set:  [1.1500999999999999, 1.3025, 5.8716999999999997, 4.4135999999999997, 2.7067000000000001, 3.5754000000000001, 2.6019000000000001, 0.73760000000000003]


In [29]:
MSE_LinR=[]
Lin_R=[]
for data in Set:
    x = data.drop(['domain1_score', 'essay','essay_set'], axis=1)
#   y = df['domain1_score']
#   x = df_normalized.drop(['domain1_score'],axis=1)
# # df['A']=df['A'].fillna(0.0).astype(int)
    y = data['domain1_score'].fillna(0.0).astype(int)
    x = np.array(x)
    y = np.array(y)
    where_are_NaNs = np.isnan(x)
    x[where_are_NaNs] = 0
    where_are_NaNs = np.isnan(y)
    y[where_are_NaNs] = 0
    
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    for train_index, test_index in kfold.split(x, y):
        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
#     print (X_train)
#     print (y_train)
    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)
    y_pred = lin_reg.predict(X_test)
#     print('\nLinear regression classifier accuracy:', linear_reg.score(X_test, y_test))
    Lin_R.append( round(lin_reg.score(X_test, y_test),4) )
    MSE_LinR.append( round(metrics.mean_squared_error(y_test,y_pred),4 ) )
print ("Linear Regression Accuracy for each set: ",Lin_R)
print ("Linear Regression MSE for each set: ",MSE_LinR)

/home/ayushi/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Linear Regression Accuracy for each set:  [0.66879999999999995, 0.53269999999999995, 0.46739999999999998, 0.59519999999999995, 0.68059999999999998, 0.55989999999999995, 0.43480000000000002, 0.40329999999999999]
Linear Regression MSE for each set:  [0.61509999999999998, 0.87350000000000005, 4.1479999999999997, 3.7938999999999998, 1.9155, 2.6093999999999999, 1.8859999999999999, 0.44750000000000001]


In [30]:
SVR=[]
MSE_SVR=[]
for data in Set:
    x = data.drop(['domain1_score', 'essay','essay_id','essay_set'], axis=1)
#   y = df['domain1_score']
#   x = df_normalized.drop(['domain1_score'],axis=1)
# # df['A']=df['A'].fillna(0.0).astype(int)
    y = data['domain1_score'].fillna(0.0).astype(int)
    x = np.array(x)
    y = np.array(y)
    where_are_NaNs = np.isnan(x)
    x[where_are_NaNs] = 0
    where_are_NaNs = np.isnan(y)
    y[where_are_NaNs] = 0
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    for train_index, test_index in kfold.split(x, y):
        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
    clf = svm.SVR(kernel="linear")
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    SVR.append(round(clf.score(X_test, y_test),4))
    MSE_SVR.append( round(metrics.mean_squared_error(y_test,y_pred),4 ) )
print ("SVM Regressor linear Accuracy for each set: ",SVR)
print ("SVM Regressor linear MSE for each set: ",MSE_SVR)

/home/ayushi/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


SVM Regressor linear Accuracy for each set:  [0.66739999999999999, 0.52129999999999999, 0.45029999999999998, 0.59179999999999999, 0.68440000000000001, 0.55010000000000003, 0.43409999999999999, 0.38550000000000001]
SVM Regressor linear MSE for each set:  [0.61770000000000003, 0.89470000000000005, 4.2808000000000002, 3.8262, 1.8929, 2.6678999999999999, 1.8883000000000001, 0.46089999999999998]


In [31]:
SVR2=[]
MSE_SVR2=[]
for data in Set:
    x = data.drop(['domain1_score', 'essay','essay_id','essay_set'], axis=1)
#   y = df['domain1_score']
#   x = df_normalized.drop(['domain1_score'],axis=1)
# # df['A']=df['A'].fillna(0.0).astype(int)
    y = data['domain1_score'].fillna(0.0).astype(int)
    x = np.array(x)
    y = np.array(y)
    where_are_NaNs = np.isnan(x)
    x[where_are_NaNs] = 0
    where_are_NaNs = np.isnan(y)
    y[where_are_NaNs] = 0
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    for train_index, test_index in kfold.split(x, y):
        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
    clf = svm.SVR(kernel="rbf")
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    SVR2.append(clf.score(X_test, y_test))
    MSE_SVR2.append( round(metrics.mean_squared_error(y_test,y_pred),4 ) )
print ("SVM Regressor RBF Accuracy for each set: ",SVR2)
print ("SVM Regressor RBF MSE for each set: ",MSE_SVR2)

/home/ayushi/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


SVM Regressor RBF Accuracy for each set:  [-0.0030000000000000001, -0.0074000000000000003, 0.065699999999999995, 0.18479999999999999, 0.1115, 0.039100000000000003, 0.011900000000000001, -0.0015]
SVM Regressor RBF MSE for each set:  [1.8627, 1.883, 7.2759999999999998, 7.6406000000000001, 5.3295000000000003, 5.6981999999999999, 3.2968000000000002, 0.75119999999999998]


In [54]:
MSE_RF=[]
RF=[]
for data in Set:
    x = data.drop(['domain1_score', 'essay','essay_id','essay_set'], axis=1)
#   y = df['domain1_score']
#   x = df_normalized.drop(['domain1_score'],axis=1)
# # df['A']=df['A'].fillna(0.0).astype(int)
    y = data['domain1_score'].fillna(0.0).astype(int)
    x = np.array(x)
    y = np.array(y)
    where_are_NaNs = np.isnan(x)
    x[where_are_NaNs] = 0
    where_are_NaNs = np.isnan(y)
    y[where_are_NaNs] = 0
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    for train_index, test_index in kfold.split(x, y):
        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
    model = RandomForestRegressor(random_state=0,n_estimators=100)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)    
#     print('Random Forest Regressor Accuracy:', clf.score(X_test, y_test))
    RF.append(round(model.score(X_test, y_test),4))
    MSE_RF.append( round(metrics.mean_squared_error(y_test,y_pred),4 ) )
print ("Random Forest Regressor Accuracy: ",RF)
print ("Random Forest Regressor MSE for each set: ",MSE_RF)


/home/ayushi/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/home/ayushi/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/home/ayushi/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Random Forest Regressor Accuracy:  [0.66830000000000001, 0.61229999999999996, 0.42799999999999999, 0.61380000000000001, 0.7036, 0.53449999999999998, 0.52529999999999999, 0.39090000000000003]
Random Forest Regressor MSE for each set:  [0.61609999999999998, 0.72460000000000002, 4.4547999999999996, 3.6200999999999999, 1.778, 2.7605, 1.5838000000000001, 0.45689999999999997]


In [50]:
results=pd.DataFrame(columns=['Model','set1','set2','set3','set4','set5','set6','set7','set8'])
results = results.append(pd.Series(MSE_LR, index=['set1','set2','set3','set4','set5','set6','set7','set8']), ignore_index=True)
results = results.append(pd.Series(MSE_LinR, index=['set1','set2','set3','set4','set5','set6','set7','set8']), ignore_index=True)
results = results.append(pd.Series(MSE_SVR, index=['set1','set2','set3','set4','set5','set6','set7','set8']), ignore_index=True)
results = results.append(pd.Series(MSE_SVR2, index=['set1','set2','set3','set4','set5','set6','set7','set8']), ignore_index=True)
results = results.append(pd.Series(MSE_RF, index=['set1','set2','set3','set4','set5','set6','set7','set8']), ignore_index=True)
results.Model=['Logistic Regression','Linear Regression','SVR(linear)','SVR(rbf)','Random Forest']

In [51]:
results.head()

,Model,set1,set2,set3,set4,set5,set6,set7,set8
0,Logistic Regression,1.1501,1.3025,5.8717,4.4136,2.7067,3.5754,2.6019,0.7376
1,Linear Regression,0.6151,0.8735,4.1480,3.7939,1.9155,2.6094,1.8860,0.4475
2,SVR(linear),0.6177,0.8947,4.2808,3.8262,1.8929,2.6679,1.8883,0.4609
3,SVR(rbf),1.8627,1.8830,7.2760,7.6406,5.3295,5.6982,3.2968,0.7512
4,Random Forest,0.6161,0.7246,4.4548,3.6201,1.7780,2.7605,1.5838,0.4569


In [52]:
from pandas import ExcelWriter
writer = ExcelWriter('Results_setwise.xlsx')
results.to_excel(writer,'Sheet1',index=False)
writer.save()

## LSTM

In [37]:
X = pd.read_excel('test2.xlsx')
# X = X.drop(X.columns[0:2],axis=1)
y = pd.DataFrame(X['domain1_score'])
X.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",6.666667
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",7.500000
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",5.833333
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",8.333333
4,5,1,"Dear @LOCATION1, I know having computers has a...",6.666667


In [38]:
Set=[]

for i in range(1,9):
    Set.append(X[X['essay_set']==i]) 
print (Set)

[      essay_id  essay_set                                              essay  \
0            1          1  Dear local newspaper, I think effects computer...   
1            2          1  Dear @CAPS1 @CAPS2, I believe that using compu...   
2            3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...   
3            4          1  Dear Local Newspaper, @CAPS1 I have found that...   
4            5          1  Dear @LOCATION1, I know having computers has a...   
5            6          1  Dear @LOCATION1, I think that computers have a...   
6            7          1  Did you know that more and more people these d...   
7            8          1  @PERCENT1 of people agree that computers make ...   
8            9          1  Dear reader, @ORGANIZATION1 has had a dramatic...   
9           10          1  In the @LOCATION1 we have the technology of a ...   
10          11          1  Dear @LOCATION1, @CAPS1 people acknowledge the...   
11          12          1  Dear @CAPS1 

In [39]:
def get_model():
    model = Sequential([
        # 2D tensor for first layer: 1 timestep and 300 features
        LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True),
        LSTM(64, recurrent_dropout=0.4),
        Dropout(0.5),
        Dense(1, activation='relu')
    ])

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

In [40]:
def essay_to_list(essay):
    # Remove the tags
    essay = re.sub("[^a-zA-Z]", " ", essay)
    words = essay.lower().split()
    return [w for w in words if not w in stopwords]

def essay_to_sentences(essay):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_list(raw_sentence))
    return sentences

# Generate feature vector for the words
def get_feature_vector(words, model, num_features, vec_type="sum"):
    feature_vector = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    
    max_vec =  np.zeros((num_features,),dtype="float32")
    min_vec =  np.ones((num_features,),dtype="float32")

    for word in words:
        if word in index2word_set:
            num_words += 1
            max_vec = np.maximum(model[word], feature_vector)
            min_vec = np.minimum(model[word], feature_vector)
            feature_vector = np.add(feature_vector, model[word]) 
    
    # return min vector + max vector
    if vec_type == "min+max":
        return np.add(min_vec, max_vec) 
    
    # average of vectors
    elif vec_type == "average":
        return np.divide(feature_vector, num_words)

    # default: return sum of word2vec vectors
    return feature_vector

# Generate word vectors from the mdoel
def generate_essay_vectors(essays, model, num_features, vec_type="sum"):
    essayfeature_vectors = np.zeros((len(essays),num_features),dtype="float32")
    for (i, essay) in enumerate(essays):
        essayfeature_vectors[i] = get_feature_vector(essay, model, num_features, vec_type)
    return essayfeature_vectors

In [58]:
LSTM_mse = []
def train_model(X, y, dataset, vec_type="sum"):
    count = 1
    results = []
    
    
    for train_set, test_set in dataset:
        print("Fold #", count)
        X_test, X_train, y_test, y_train = X.iloc[test_set], X.iloc[train_set], y.iloc[test_set], y.iloc[train_set]
        
        train_essays = X_train['essay']
        test_essays = X_test['essay']
        
        sentences = []
        
        for essay in train_essays:
            sentences += essay_to_sentences(essay)
                
        # Initialize variables for word2vec model
        num_features = 300 
        min_word_count = 40
        num_workers = 4
        context = 10
        downsampling = 1e-7

        # Train the word2vec model
        model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)
        model.init_sims(replace=True)
        
        # Generate training vectors
        clean_train_essays = []
        for essay_vec in train_essays:
            clean_train_essays.append(essay_to_list(essay_vec))
        train_vectors = generate_essay_vectors(clean_train_essays, model, num_features, vec_type)
        
        # Generate test vectors
        clean_test_essays = []
        for essay_vec in test_essays:
            clean_test_essays.append(essay_to_list( essay_vec))
        test_vectors = generate_essay_vectors(clean_test_essays, model, num_features, vec_type)
        
        train_vectors = np.array(train_vectors)
        test_vectors = np.array(test_vectors)

        # Reshape the train and test vectors to 3 dimensions - 1 represents one timestamp 
        train_vectors = np.reshape(train_vectors, (train_vectors.shape[0], 1, train_vectors.shape[1]))
        test_vectors = np.reshape(test_vectors, (test_vectors.shape[0], 1, test_vectors.shape[1]))
        
        # Call the LSTM to get the score predictions 
        lstm_model = get_model()
        lstm_model.fit(train_vectors, y_train, batch_size=64, epochs=50)
        y_pred = lstm_model.predict(test_vectors)
        
        # Round the prediction to the nearest integer
        y_pred = np.around(y_pred)
        
        # Evaluate the model: quadratic kappa score of predictions against human grading
        result = cohen_kappa_score(y_test.values, y_pred, weights='quadratic')
        print("QWK: ", result)
        results.append(result)
        
        LSTM_mse.append( round(metrics.mean_squared_error(y_test.values,y_pred),4 ) )
        count += 1
    
    return results

In [64]:
from statistics import mean
Min_Max=[]
MM_mse=[]
for data in Set: 
    y=pd.DataFrame()
    X=pd.DataFrame()
    X = data
    y['domain1_score']=X['domain1_score']
    X = X.replace('NaN', 0)
    y['domain1_score']=y['domain1_score'].fillna(0.0).astype(int)
    dataset = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    dataset = dataset.split(X, y)
    results_min_max = train_model(X, y, dataset, "min+max")
    results_min_max=np.around(np.array(results_min_max).mean(),decimals=4)
    Min_Max.append(results_min_max)
    MM_mse.append(mean(LSTM_mse))
    print("MSE=",mean(LSTM_mse))
    print("Average Quadratic Weighted Kappa after 5-fold cross validation for min + max word2vec ",results_min_max)
    

/home/ayushi/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Fold # 1


/home/ayushi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/ayushi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/ayushi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_103 (LSTM)              (None, 1, 300)            721200    
_________________________________________________________________
lstm_104 (LSTM)              (None, 64)                93440     
_________________________________________________________________
dropout_52 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_52 (Dense)             (None, 1)                 65        
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
1423/1423 [==============================] - 10s 7ms/step - loss: 8.9540 - mean_absolute_error: 2.1833
Epoch 2/50
1423/1423 [==============================] - 1s 560us/step - loss: 2.0374 - mean_absolute_error: 1.1259
Epoch 3/50
1423/1423 [=================

1423/1423 [==============================] - 1s 578us/step - loss: 1.5969 - mean_absolute_error: 0.9957
Epoch 7/50
1423/1423 [==============================] - 1s 695us/step - loss: 1.6172 - mean_absolute_error: 1.0016
Epoch 8/50
1423/1423 [==============================] - 1s 573us/step - loss: 1.5597 - mean_absolute_error: 0.9882
Epoch 9/50
1423/1423 [==============================] - 1s 595us/step - loss: 1.5367 - mean_absolute_error: 0.9766
Epoch 10/50
1423/1423 [==============================] - 1s 659us/step - loss: 1.5467 - mean_absolute_error: 0.9778
Epoch 11/50
1423/1423 [==============================] - 1s 596us/step - loss: 1.4765 - mean_absolute_error: 0.9518
Epoch 12/50
1423/1423 [==============================] - 1s 600us/step - loss: 1.4404 - mean_absolute_error: 0.9370
Epoch 13/50
1423/1423 [==============================] - 1s 675us/step - loss: 1.4028 - mean_absolute_error: 0.9328
Epoch 14/50
1423/1423 [==============================] - 1s 643us/step - loss: 1.4127 -

1428/1428 [==============================] - 1s 568us/step - loss: 1.3717 - mean_absolute_error: 0.9257
Epoch 19/50
1428/1428 [==============================] - 1s 556us/step - loss: 1.3944 - mean_absolute_error: 0.9309
Epoch 20/50
1428/1428 [==============================] - 1s 553us/step - loss: 1.2250 - mean_absolute_error: 0.8761
Epoch 21/50
1428/1428 [==============================] - 1s 558us/step - loss: 1.2954 - mean_absolute_error: 0.8951
Epoch 22/50
1428/1428 [==============================] - 1s 554us/step - loss: 1.3800 - mean_absolute_error: 0.9240
Epoch 23/50
1428/1428 [==============================] - 1s 562us/step - loss: 1.2405 - mean_absolute_error: 0.8694
Epoch 24/50
1428/1428 [==============================] - 1s 570us/step - loss: 1.2456 - mean_absolute_error: 0.8905
Epoch 25/50
1428/1428 [==============================] - 1s 573us/step - loss: 1.2613 - mean_absolute_error: 0.8794
Epoch 26/50
1428/1428 [==============================] - 1s 550us/step - loss: 1.268

1428/1428 [==============================] - 1s 574us/step - loss: 1.1544 - mean_absolute_error: 0.8416
Epoch 31/50
1428/1428 [==============================] - 1s 560us/step - loss: 1.1252 - mean_absolute_error: 0.8344
Epoch 32/50
1428/1428 [==============================] - 1s 558us/step - loss: 1.2010 - mean_absolute_error: 0.8639
Epoch 33/50
1428/1428 [==============================] - 1s 564us/step - loss: 1.2022 - mean_absolute_error: 0.8651
Epoch 34/50
1428/1428 [==============================] - 1s 553us/step - loss: 1.1868 - mean_absolute_error: 0.8443
Epoch 35/50
1428/1428 [==============================] - 1s 563us/step - loss: 1.0988 - mean_absolute_error: 0.8311
Epoch 36/50
1428/1428 [==============================] - 1s 551us/step - loss: 1.1296 - mean_absolute_error: 0.8430
Epoch 37/50
1428/1428 [==============================] - 1s 570us/step - loss: 1.1779 - mean_absolute_error: 0.8479
Epoch 38/50
1428/1428 [==============================] - 1s 572us/step - loss: 1.092

1430/1430 [==============================] - 1s 639us/step - loss: 1.1005 - mean_absolute_error: 0.8238
Epoch 44/50
1430/1430 [==============================] - 1s 604us/step - loss: 1.0684 - mean_absolute_error: 0.8130
Epoch 45/50
1430/1430 [==============================] - 1s 625us/step - loss: 1.1309 - mean_absolute_error: 0.8284
Epoch 46/50
1430/1430 [==============================] - 1s 592us/step - loss: 1.0821 - mean_absolute_error: 0.8171
Epoch 47/50
1430/1430 [==============================] - 1s 634us/step - loss: 0.9752 - mean_absolute_error: 0.7778
Epoch 48/50
1430/1430 [==============================] - 1s 641us/step - loss: 0.9887 - mean_absolute_error: 0.7904
Epoch 49/50
1430/1430 [==============================] - 1s 637us/step - loss: 0.9918 - mean_absolute_error: 0.7882
Epoch 50/50
1430/1430 [==============================] - 1s 631us/step - loss: 1.1127 - mean_absolute_error: 0.8180
QWK:  0.631887590545
MSE= 1.17927
Average Quadratic Weighted Kappa after 5-fold cros

KeyboardInterrupt: 

In [65]:
print (Min_Max)
print (MM_mse)

[0.65290000000000004]
[1.17927]


In [7]:
Sum=[]
for data in Set: 
#     print(data.head())
    y=pd.DataFrame()
    X=pd.DataFrame()
    X = data
    y['domain1_score']=X['domain1_score']
    X = X.replace('NaN', 0)
    y['domain1_score']=y['domain1_score'].fillna(0.0).astype(int)
#     print(X.head())
#     print(y.head())
    dataset = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    dataset = dataset.split(X, y)
    results_sum = train_model(X, y, dataset)
    results_sum=np.around(np.array(results_sum).mean(),decimals=4)
    Sum.append(results_sum)
    print("Average Quadratic Weighted Kappa after 5-fold cross validation for sum word2vec ",results_sum)

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Fold # 1


/home/nikshubha/.local/lib/python3.5/site-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/nikshubha/.local/lib/python3.5/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/nikshubha/.local/lib/python3.5/site-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 300)            721200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
1423/1423 [===========

Epoch 1/50
1423/1423 [==============================] - 2s 2ms/step - loss: 8.6724 - mean_absolute_error: 2.1621
Epoch 2/50
1423/1423 [==============================] - 1s 476us/step - loss: 1.9768 - mean_absolute_error: 1.1068
Epoch 3/50
1423/1423 [==============================] - 1s 484us/step - loss: 1.7857 - mean_absolute_error: 1.0516
Epoch 4/50
1423/1423 [==============================] - 1s 483us/step - loss: 1.7063 - mean_absolute_error: 1.0254
Epoch 5/50
1423/1423 [==============================] - 1s 500us/step - loss: 1.6579 - mean_absolute_error: 0.9908
Epoch 6/50
1423/1423 [==============================] - 1s 491us/step - loss: 1.7346 - mean_absolute_error: 1.0381
Epoch 7/50
1423/1423 [==============================] - 1s 500us/step - loss: 1.6439 - mean_absolute_error: 1.0006
Epoch 8/50
1423/1423 [==============================] - 1s 496us/step - loss: 1.5669 - mean_absolute_error: 0.9801
Epoch 9/50
1423/1423 [==============================] - 1s 496us/step - loss: 1.52

...

1260/1260 [==============================] - 1s 950us/step - loss: 1.4191 - mean_absolute_error: 0.9384
Epoch 45/50
1260/1260 [==============================] - 1s 956us/step - loss: 1.4345 - mean_absolute_error: 0.9491
Epoch 46/50
1260/1260 [==============================] - 1s 873us/step - loss: 1.4582 - mean_absolute_error: 0.9442
Epoch 47/50
1260/1260 [==============================] - 1s 723us/step - loss: 1.3549 - mean_absolute_error: 0.9106
Epoch 48/50
1260/1260 [==============================] - 1s 700us/step - loss: 1.4055 - mean_absolute_error: 0.9361
Epoch 49/50
1260/1260 [==============================] - 1s 722us/step - loss: 1.3096 - mean_absolute_error: 0.8982
Epoch 50/50
1260/1260 [==============================] - 1s 712us/step - loss: 1.3302 - mean_absolute_error: 0.9030
QWK:  0.68662718858555
Average Quadratic Weighted Kappa after 5-fold cross validation for sum word2vec  0.6579
Fold # 1


/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/home/nikshubha/.local/lib/python3.5/site-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/nikshubha/.local/lib/python3.5/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/nikshubha/.local/lib/python3.5/site-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_71 (LSTM)               (None, 1, 300)            721200    
_________________________________________________________________
lstm_72 (LSTM)               (None, 64)                93440     
_________________________________________________________________
dropout_36 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 65        
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
577/577 [==============================] - 14s 24ms/step - loss: 12.0938 - mean_absolute_error: 2.8611
Epoch 2/50
577/577 [==============================] - 0s 522us/step - loss: 1.8653 - mean_absolute_error: 1.0896
Epoch 3/50
577/577 [=====================

578/578 [==============================] - 0s 657us/step - loss: 1.0464 - mean_absolute_error: 0.7997
Epoch 22/50
578/578 [==============================] - 0s 648us/step - loss: 0.9055 - mean_absolute_error: 0.7529
Epoch 23/50
578/578 [==============================] - 0s 611us/step - loss: 0.9674 - mean_absolute_error: 0.7632
Epoch 24/50
578/578 [==============================] - 0s 617us/step - loss: 0.9565 - mean_absolute_error: 0.7729
Epoch 25/50
578/578 [==============================] - 0s 713us/step - loss: 0.8945 - mean_absolute_error: 0.7379
Epoch 26/50
578/578 [==============================] - 0s 583us/step - loss: 0.8607 - mean_absolute_error: 0.7447
Epoch 27/50
578/578 [==============================] - 0s 587us/step - loss: 0.8347 - mean_absolute_error: 0.7199
Epoch 28/50
578/578 [==============================] - 0s 583us/step - loss: 0.8826 - mean_absolute_error: 0.7373
Epoch 29/50
578/578 [==============================] - 0s 603us/step - loss: 0.8533 - mean_absolute_

582/582 [==============================] - 0s 633us/step - loss: 0.7433 - mean_absolute_error: 0.6772
Epoch 50/50
582/582 [==============================] - 0s 637us/step - loss: 0.7883 - mean_absolute_error: 0.7095
QWK:  0.5268289236638525
Average Quadratic Weighted Kappa after 5-fold cross validation for sum word2vec  0.5076


In [8]:
print(Sum)

[0.6655, 0.683, 0.5929, 0.7757, 0.7939, 0.7729, 0.6579, 0.5076]


In [17]:
Min_Max=[0.6932, 0.6717, 0.5925, 0.747, 0.7897, 0.7866, 0.6623, 0.4241]

In [18]:
results_lstm=pd.DataFrame(columns=['Model','set1','set2','set3','set4','set5','set6','set7','set8'])
results_lstm = results_lstm.append(pd.Series(Min_Max, index=['set1','set2','set3','set4','set5','set6','set7','set8']), ignore_index=True)
results_lstm = results_lstm.append(pd.Series(Sum, index=['set1','set2','set3','set4','set5','set6','set7','set8']), ignore_index=True)
results_lstm.Model=['Average Quadratic Weighted Kappa after 5-fold cross validation for min+max word2vec','Average Quadratic Weighted Kappa after 5-fold cross validation for sum word2vec ']
print (results_lstm.head())

                                               Model    set1    set2    set3  \
0  Average Quadratic Weighted Kappa after 5-fold ...  0.6932  0.6717  0.5925   
1  Average Quadratic Weighted Kappa after 5-fold ...  0.6655  0.6830  0.5929   

     set4    set5    set6    set7    set8  
0  0.7470  0.7897  0.7866  0.6623  0.4241  
1  0.7757  0.7939  0.7729  0.6579  0.5076  


In [20]:
from pandas import ExcelWriter
writer = ExcelWriter('Results_LSTM_setwise.xlsx')
results_lstm.to_excel(writer,'Sheet1',index=False)
writer.save()